<a href="https://colab.research.google.com/github/BritCrit/AOR-List-Comparison/blob/main/AOR_Zip_Comparison_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter
from google.colab import files
import pandas as pd

region_select = 'SE' # @param ['NC', 'NE', 'SC', 'SE', 'W']

# Prompt the user to upload the files
print("Please upload the current dealer zip code list as an Excel file.")
uploaded_df1 = files.upload()
df1_filename = list(uploaded_df1.keys())[0]

print("Please upload the prior dealer zip code list as an Excel file.")
uploaded_df2 = files.upload()
df2_filename = list(uploaded_df2.keys())[0]


# Load the Excel files into pandas DataFrames
df1 = pd.read_excel('/content/' + df1_filename, skiprows=1)
df2 = pd.read_excel('/content/' + df2_filename, skiprows=1)
# Compare the DataFrames
diff_rows = pd.concat([df1, df2]).drop_duplicates(keep=False)
# Print the differences
diff_rows = diff_rows.reset_index()

# You can now work with the diff_rows DataFrame, e.g., print it or save it to a new file
diff_rows


# Find rows only in df1 (added in October)
added_rows = df1[~df1.isin(df2).all(1)].copy()
added_rows['Status'] = 'Added'

# Find rows only in df2 (removed in October)
removed_rows = df2[~df2.isin(df1).all(1)].copy()
removed_rows['Status'] = 'Removed'

# Combine the added and removed rows
diff_rows = pd.concat([added_rows, removed_rows])
diff_rows = diff_rows.reset_index(drop=True)

# Save the differences to a new Excel file
# diff_rows.to_excel('October Changes AOR List 11.6.24 Test File.xlsx', index=False)

# Calculate the percentage of added rows
if len(df1) > 0:
  percentage_added = (len(added_rows) / len(df1)) * 100
else:
  percentage_added = 0

print(f"Percentage of rows added this month: {percentage_added:.2f}%")


impacted_dealers = diff_rows.BAC.unique()

# Filter df1 based on impacted_dealers
filtered_df1 = df1[df1['BAC'].isin(impacted_dealers)]

# Now filtered_df1 contains only the rows where the 'BAC' value is present in impacted_dealers
filtered_df1

# Assuming 'Region' and 'Market' are columns in your DataFrame
report = filtered_df1.groupby(['Region', 'DMA Name'])['BAC'].nunique().reset_index()

# Rename the 'BAC' column to 'Unique BAC Count'
report = report.rename(columns={'BAC': 'Count Dealers with Zip Changes'})

# Print or save the report
print("--------------------------------------")
print("Showing Number of Regional Changes to Dealers")
print(report.groupby("Region")["Count Dealers with Zip Changes"].sum())
print("--------------------------------------")
print("Selected Region Changes by DMA")
print(report[report.Region == region_select])
print("--------------------------------------")
filtered_df1['Zip Code'] = filtered_df1['Zip Code'].astype(str).str.zfill(5)  # Pad with leading zeros to 5 digits

print("--------------------------------------")
print("Collecting Impacted Dealers and Associated Zip Codes")
bac_zips = df1.groupby('BAC')['Zip Code'].agg(list).reset_index()

zips_to_update =  pd.merge(filtered_df1[['BAC', 'Division', 'Market', 'TA', 'Dealer Name', 'Address', 'City',
       'State', 'Zone', 'Region', 'DMA Name', 'LMA Description',
       'LMA Indicator']].drop_duplicates(), bac_zips, on='BAC', how='left').reset_index(drop=True)

zips_to_update['Zip Code'] = zips_to_update['Zip Code'].apply(lambda x: ','.join(str(i) for i in x) if isinstance(x, list) else str(x))

zips_to_update = zips_to_update[['Region', 'DMA Name', 'Dealer Name', 'BAC', 'Division', 'Market', 'TA',  'Address', 'City',
       'State', 'Zone', 'LMA Description',
       'LMA Indicator', 'Zip Code']]

zips_to_update = zips_to_update.sort_values(['Region', 'DMA Name', 'Dealer Name']).reset_index(drop=True)

zips_to_update['Zip Code'] = zips_to_update['Zip Code'].astype(str)

print("--------------------------------------")
print("Creating Regional Specific List of Dealers with Changes")

zips_to_update_region = zips_to_update[zips_to_update['Region'] == 'SE']

zips_to_update.to_excel("output.xlsx", engine='xlsxwriter', index=False)

print("--------------------------------------")
print("Downloading Files")
print("--------------------------------------")
print("If only one set of files downloaded, check options to allow multiple downloads")
print("--------------------------------------")

# Save zips_to_update DataFrame to Excel and download
report.to_excel('region_dealer_count_changes_report.xlsx', index=False)
zips_to_update.to_excel('all_regions_zips_to_update.xlsx', engine='xlsxwriter', index=False)
zips_to_update_region.to_excel(f"{region_select} zips_to_update_region.xlsx", engine='xlsxwriter' ,index=False)
files.download('all_regions_zips_to_update.xlsx')
files.download('region_dealer_count_changes_report.xlsx')
files.download(f"{region_select} zips_to_update_region.xlsx")